# Etapa de pre-processamento

Transformações efetuadas:
- Seleção das features
- Normalização das features contínuas
- Split entre train/test/eval

In [ ]:
# Imports
%pip install -r requirements.txt -q

import pandas as pd
from google.cloud import bigquery
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


Note: you may need to restart the kernel to use updated packages.


In [27]:
# Load
PROJECT_ID = "ai-platform-mockup"
INPUT_DATASET_ID = "staging"
TABLE_ID = "p1_county_natality"

client = bigquery.Client(project=PROJECT_ID)

query = f"SELECT * FROM  `{PROJECT_ID}.{INPUT_DATASET_ID}.{TABLE_ID}`"
df = client.query(query).to_dataframe()

In [28]:
# Selection
selected_columns = [
    "Births",
    "Ave_Age_of_Mother",
    "Ave_OE_Gestational_Age_Wks",
    "Ave_LMP_Gestational_Age_Wks",
    "Ave_Birth_Weight_gms",
    "Ave_Pre_pregnancy_BMI",
    "Ave_Number_of_Prenatal_Wks"
]
df = df[selected_columns]

In [29]:
# Normalization
continuous_columns = [
    "Ave_Age_of_Mother",
    "Ave_OE_Gestational_Age_Wks",
    "Ave_LMP_Gestational_Age_Wks",
    "Ave_Birth_Weight_gms",
    "Ave_Pre_pregnancy_BMI",
    "Ave_Number_of_Prenatal_Wks"
]

df[continuous_columns] = MinMaxScaler().fit_transform(df[continuous_columns])

In [30]:
# Split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
test_df, eval_df = train_test_split(test_df, test_size=0.5, random_state=42)

train_df["split"] = "train"
test_df["split"] = "test"
eval_df["split"] = "eval"

combined_df = pd.concat([train_df, test_df, eval_df], ignore_index=True)


In [31]:
# Save to BigQuery
OUTPUT_DATASET_ID = "preprocessed"
table_full_id = f"{PROJECT_ID}.{OUTPUT_DATASET_ID}.{TABLE_ID}"
combined_df.to_gbq(table_full_id, project_id=PROJECT_ID, if_exists="replace")

C:\Users\Roberto\AppData\Local\Temp\ipykernel_76288\213831441.py:4: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  combined_df.to_gbq(table_id, project_id=PROJECT_ID, if_exists="replace")
100%|██████████| 1/1 [00:00<00:00, 18157.16it/s]
